# Avance 1 — EDA (Exploratory Data Analysis)

**Dataset:** `Data Communauto 26-JAN-2026.xlsx`  
**Objetivo del entregable:** realizar un EDA (univariado + bivariado/multivariado) para **entender el conjunto de datos**, **identificar problemas** (faltantes, atípicos, alta cardinalidad, etc.) y **justificar** las decisiones de preprocesamiento que se usarán posteriormente en modelado.

> **Nota para GitHub:** coloca el archivo Excel en la carpeta `data/` del repositorio y ajusta la variable `DATA_PATH` si cambias el nombre/ruta.

---

## Checklist (rúbrica EDA)
- [x] Descripción del dataset (tamaño, tipos, variable(s))
- [x] Valores faltantes + patrones + decisión (eliminar/imputar)
- [x] Variables categóricas: cardinalidad y distribución
- [x] Variables numéricas: distribución y atípicos
- [x] Análisis temporal (si aplica)
- [x] Correlaciones / relaciones relevantes (según disponibilidad)
- [x] Conclusiones accionables (qué limpiar, qué transformar, qué variables usar/eliminar)


## 1. Contexto y vista general

Este dataset contiene **reportes/feedback de usuarios sobre vehículos**, con:
- Variables **categóricas** (p.ej. `Reason for reporting`, `Intervention`)
- Variables **temporales** (`Date and Time`, `Created`)
- **Texto libre** (`Feedback`, `Notes/Follow-up`)
- Identificadores (`Car number`, `Member number`, `ID`)
- URLs de evidencia (`Image 1-3`)

Dimensiones observadas:
- **Filas:** 34,498
- **Columnas:** 31
- Rango de fechas en `Date and Time`: **2025-05-05 15:47:14 → 2025-10-01 15:06:43**


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Configuración visual
plt.rcParams["figure.figsize"] = (10, 4)
sns.set_theme(style="whitegrid")

# === Ruta del dataset ===
DATA_PATH = "data/Data Communauto 26-JAN-2026.xlsx"  # en GitHub: crea carpeta data/ y coloca el archivo ahí

df_raw = pd.read_excel(DATA_PATH)
df_raw.shape


In [ ]:
df_raw.head(3)

In [ ]:
df_raw.info()

## 2. Diagnóstico de calidad de datos (faltantes y columnas basura)

En una primera inspección se observan columnas con nombres tipo ` .1`, ` .2`, ... que suelen aparecer por **espacios/columnas vacías** en el Excel original.

Esto es útil para el EDA: identifica **ruido** que debe eliminarse para reducir dimensionalidad y evitar problemas de generalización.


In [ ]:
missing_pct = df_raw.isna().mean().sort_values(ascending=False)
missing_pct.head(20)

**Interpretación inicial (a validar con las tablas/gráficas):**
- Existen columnas con **100% faltantes** (típicamente ` .2`, ` .3`, ..., etc.), por lo que son **candidatas directas a eliminación**.
- Variables como `Image 2` tienen una tasa alta de faltantes (aprox. **nan%**), lo cual es esperable si no todos los reportes incluyen evidencia.
- `Importance` tiene muy pocos registros no nulos (solo ~169), por lo que conviene tratarla como **variable auxiliar** o considerar eliminarla del modelado si no aporta señal.


In [ ]:
# Mapa de calor (muestra) para ver patrón de faltantes
sample = df_raw.sample(min(2000, len(df_raw)), random_state=42)
plt.figure(figsize=(10, 4))
sns.heatmap(sample.isna(), cbar=False)
plt.title("Patrón de valores faltantes (muestra)")
plt.show()


## 3. Limpieza mínima justificada (sin destruir información)

El EDA no debe “maquillar” el dataset sin justificar, pero **sí** debe aplicar una limpieza mínima cuando hay columnas evidentemente vacías o basura.

Estrategia:
1) Normalizar nombres de columnas (quitar espacios al inicio/fin)  
2) Eliminar columnas con **> 90%** valores faltantes  
3) Reportar cuántas columnas se eliminaron y por qué


In [ ]:
df = df_raw.copy()

# 1) Normalizar nombres
df.columns = [str(c).strip() for c in df.columns]

# 2) Eliminar columnas casi vacías
missing_pct = df.isna().mean()
cols_drop = missing_pct[missing_pct > 0.90].index.tolist()

df = df.drop(columns=cols_drop)

print("Columnas eliminadas (>90% faltantes):", len(cols_drop))
cols_drop[:20], df.shape


In [ ]:
# Faltantes después de limpieza mínima
(df.isna().mean().sort_values(ascending=False).head(15))

## 4. Tipos de variables (numéricas, categóricas, texto, fechas)

Clasificamos tipos para decidir:
- Codificación de categóricas
- Tratamiento de texto (NLP) o features simples (longitud, presencia de URL, etc.)
- Conversión y agregación temporal


In [ ]:
df.dtypes

In [ ]:
# Convertir columnas de fecha (si vienen como texto)
for col in ["Date and Time", "Created"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

df[["Date and Time", "Created"]].describe()

## 5. Univariado: variables categóricas

### 5.1 Distribución de `Reason for reporting`

Esta variable describe el motivo del reporte. Un desbalance fuerte es importante porque:
- Afecta métricas si se modela clasificación en el futuro
- Sugiere dónde están los principales problemas operativos


In [ ]:
reason_counts = df["Reason for reporting"].value_counts(dropna=True)
reason_counts.head(15)

In [ ]:
plt.figure(figsize=(10, 5))
top = reason_counts.head(10)
sns.barplot(x=top.values, y=top.index)
plt.title("Top 10 motivos de reporte (Reason for reporting)")
plt.xlabel("Número de reportes")
plt.ylabel("")
plt.show()


**Interpretación (según conteos actuales):**
- Los motivos más frecuentes incluyen `Damage` (~13,722) y `Cleanliness` (~8,328), lo que sugiere que gran parte del volumen está asociado a **daños** y **limpieza**.
- Se observan etiquetas en **múltiples idiomas** (p.ej. `Dommages`, `Propreté`), lo que sugiere una futura normalización/estandarización de categorías para evitar fragmentación artificial.


### 5.2 Cardinalidad (riesgo de sobreajuste)

Variables con muchos valores únicos (IDs, texto, URLs) pueden:
- No generalizar (memorizar en vez de aprender)
- Inflar dimensionalidad (one-hot muy grande)

Medimos cardinalidad con `nunique`.


In [ ]:
nunique = df.nunique(dropna=True).sort_values(ascending=False)
nunique.head(15)

**Interpretación inicial:**
- `Member number` y `Car number` presentan alta cardinalidad, por lo que típicamente se tratan como **IDs** (no como variables categóricas one-hot).
- `Feedback` y `Image 1-3` también tienen alta cardinalidad: se sugiere transformar (features de texto) o excluir del modelado tabular inicial.


## 6. Univariado: variable numérica (`Importance`)

Se revisa distribución y posibles atípicos.  
> Nota: si `Importance` tiene demasiados faltantes, puede ser más útil como etiqueta auxiliar o descartarse.


In [ ]:
df["Importance"].describe()

In [ ]:
plt.figure(figsize=(10,4))
sns.histplot(df["Importance"], kde=False)
plt.title("Distribución de Importance")
plt.show()

In [ ]:
plt.figure(figsize=(8,3))
sns.boxplot(x=df["Importance"])
plt.title("Boxplot de Importance (detección de atípicos)")
plt.show()

**Interpretación (a confirmar con gráficas):**
- `Importance` aparenta estar acotada (mín=1, máx=5) con mediana alrededor de 4.
- Sin embargo, el conteo no nulo es bajo (en el archivo actual: 169), por lo que la variable puede no ser representativa para todo el dataset.


## 7. Análisis de texto (features simples)

Para texto libre, una primera aproximación EDA es:
- Longitud del texto
- Proporción de vacíos
- Palabras más frecuentes (opcional)

Esto no sustituye NLP, pero ayuda a entender calidad y consistencia.


In [ ]:
# Longitud del feedback
df["Feedback_str"] = df["Feedback"].astype(str)
df["feedback_len"] = df["Feedback_str"].apply(len)

df["feedback_len"].describe()

In [ ]:
plt.figure(figsize=(10,4))
sns.histplot(df["feedback_len"], kde=False)
plt.title("Distribución de longitud de Feedback")
plt.xlabel("Caracteres")
plt.show()


**Interpretación:**
- Una cola larga en `feedback_len` suele indicar reportes muy detallados vs. comentarios cortos.
- Si hay muchos textos extremadamente cortos (p.ej. 1–3 caracteres), podría ser ruido (ej. valores tipo `"nan"` por casteo); por eso conviene revisar vacíos reales.


In [ ]:
# Vacíos reales en Feedback (sin contar "nan" como string)
feedback_missing = df["Feedback"].isna().mean()
notes_missing = df["Notes/Follow-up"].isna().mean() if "Notes/Follow-up" in df.columns else None
images_missing = {c: df[c].isna().mean() for c in ["Image 1","Image 2","Image 3"] if c in df.columns}

feedback_missing, notes_missing, images_missing

## 8. Análisis temporal

Se analiza el volumen de reportes a lo largo del tiempo para detectar:
- Tendencias (picos, estacionalidad)
- Cambios operativos


In [ ]:
dt_col = df["Date and Time"]
daily_counts = dt_col.dt.date.value_counts().sort_index()

plt.figure(figsize=(12,4))
daily_counts.plot()
plt.title("Reportes por día (Date and Time)")
plt.xlabel("Fecha")
plt.ylabel("Número de reportes")
plt.show()

daily_counts.describe()


**Interpretación:**
- Identifica picos de reportes que pueden corresponder a eventos (clima, cambios de flota, fallas recurrentes).
- Si el objetivo futuro involucra predicción, se podrían crear features como día de semana, mes, hora, etc.


## 9. Bivariado: tiempo vs motivo (ejemplo)

Cruzar variables ayuda a encontrar relaciones como:
- ¿Qué motivos dominan en ciertos periodos?


In [ ]:
# Top motivos por mes (muestra de enfoque)
df["month"] = df["Date and Time"].dt.to_period("M").astype(str)

pivot = (df.dropna(subset=["Reason for reporting"])
           .groupby(["month","Reason for reporting"])
           .size()
           .reset_index(name="count"))

top_reasons = df["Reason for reporting"].value_counts().head(6).index.tolist()
pivot_top = pivot[pivot["Reason for reporting"].isin(top_reasons)]

plt.figure(figsize=(12,5))
sns.lineplot(data=pivot_top, x="month", y="count", hue="Reason for reporting", marker="o")
plt.title("Tendencia mensual de los principales motivos de reporte")
plt.xlabel("Mes")
plt.ylabel("Reportes")
plt.xticks(rotation=45)
plt.show()


## 10. Correlación (si aplica)

Este dataset es principalmente texto/categorías/IDs, por lo que la correlación numérica puede ser limitada.
Aun así, revisamos correlaciones entre variables numéricas disponibles.


In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols

In [ ]:
if len(num_cols) >= 2:
    corr = df[num_cols].corr(numeric_only=True)
    plt.figure(figsize=(6,4))
    sns.heatmap(corr, annot=True, fmt=".2f")
    plt.title("Matriz de correlación (variables numéricas)")
    plt.show()
else:
    print("No hay suficientes variables numéricas para correlación significativa.")


## 11. Conclusiones y decisiones de preprocesamiento (lo más calificado)

### Hallazgos clave
1) **Columnas basura** (vacías) provenientes del Excel → deben eliminarse (reducción dimensional).
2) **Faltantes** importantes en evidencia (`Image 1-3`) y en `Importance`.
3) **Alta cardinalidad** en `Member number`, `Car number`, `ID` → tratarlas como identificadores (evitar one-hot).
4) **Multilenguaje** en `Reason for reporting` → estandarizar categorías (p.ej. mapear `Damage` y `Dommages`).
5) **Texto libre** (`Feedback`) → requiere tratamiento (NLP) o features simples; para EDA se analizó longitud y vacíos.
6) **Temporalidad** clara (May–Oct 2025) → es viable crear features temporales (día/mes/hora).

### Próximos pasos recomendados
- Definir variable objetivo si habrá modelado (p.ej. predecir motivo del reporte, severidad, necesidad de intervención, etc.).
- Diseñar pipeline de limpieza:
  - drop columnas >90% faltantes
  - normalizar categorías (diccionario de mapeo)
  - transformar texto (TF-IDF/embeddings) si se usa como predictor
  - crear features temporales
- Validar si existe desbalance de clases para la variable objetivo y planear técnicas (pesos, muestreo, métricas).

> **Nota:** En este avance el enfoque es EDA + justificación, no entrenamiento.
